In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/bert_withstopwordchangelevels_50_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,0.021147,-0.530477,-0.624731,1.005194,-0.642407,-1.079751,0.389008,-0.168904,-0.757797,...,0.588148,0.254079,0.309489,0.438753,-0.033048,0.637591,0.869479,0.205174,1.006595,0.846551
1,1,0.105614,-1.102952,-0.442954,0.446945,-1.175067,-1.423418,-0.318435,-0.436999,-0.334121,...,0.486520,0.029947,0.712149,0.212562,-0.283283,-0.499763,0.594637,0.085103,0.737417,-0.028257
2,2,0.714089,-0.580954,-1.211074,1.017179,0.498596,-0.752542,0.440776,0.569260,-0.734171,...,1.038670,1.514075,0.021501,0.345835,-0.315350,0.382593,0.576472,1.673322,0.715242,0.754557
3,3,-0.629240,0.987672,0.713336,0.017152,-1.224736,0.978825,-1.077086,-0.765753,1.451566,...,-0.798350,-1.364312,1.015312,-0.799690,1.429670,-0.331538,-1.834699,-0.909356,-1.041505,-1.733988
4,4,0.860277,-0.535206,-1.623034,1.545206,-0.341238,-1.399060,0.663807,1.578156,-1.261302,...,1.376087,1.202711,-0.967703,1.911900,-1.158021,0.936117,0.980765,0.633990,1.244317,2.191585


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.021147,-0.530477,-0.624731,1.005194,-0.642407,-1.079751,0.389008,-0.168904,-0.757797,-0.479697,...,0.588148,0.254079,0.309489,0.438753,-0.033048,0.637591,0.869479,0.205174,1.006595,0.846551
1,0.105614,-1.102952,-0.442954,0.446945,-1.175067,-1.423418,-0.318435,-0.436999,-0.334121,-0.239124,...,0.486520,0.029947,0.712149,0.212562,-0.283283,-0.499763,0.594637,0.085103,0.737417,-0.028257
2,0.714089,-0.580954,-1.211074,1.017179,0.498596,-0.752542,0.440776,0.569260,-0.734171,-0.968783,...,1.038670,1.514075,0.021501,0.345835,-0.315350,0.382593,0.576472,1.673322,0.715242,0.754557
3,-0.629240,0.987672,0.713336,0.017152,-1.224736,0.978825,-1.077086,-0.765753,1.451566,0.998886,...,-0.798350,-1.364312,1.015312,-0.799690,1.429670,-0.331538,-1.834699,-0.909356,-1.041505,-1.733988
4,0.860277,-0.535206,-1.623034,1.545206,-0.341238,-1.399060,0.663807,1.578156,-1.261302,-0.831212,...,1.376087,1.202711,-0.967703,1.911900,-1.158021,0.936117,0.980765,0.633990,1.244317,2.191585


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 50), (524, 50))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [15]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [16]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3442576415907994


**Decision Tree**

In [17]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [18]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [19]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [20]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [21]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [22]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.395902,0.395902,0.639916,0.326736,0.280534,0.280534,0.323292,0.212595


**Random Forest**

In [23]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [24]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [25]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [26]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9972    1.0000    0.9986       361
           2     1.0000    0.9982    0.9991       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         0.9992      1220
   macro avg     0.9991    0.9994    0.9992      1220
weighted avg     0.9992    0.9992    0.9992      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3293    0.1720    0.2259       157
           2     0.4611    0.7511    0.5714       237
           3     0.2321    0.1000    0.1398       130

    accuracy                         0.4160       524
   macro avg     0.3409    0.3410    0.3124       524
weighted avg     0.3648    0.4160    0.3608       524



In [27]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.395902,0.395902,0.639916,0.326736,0.280534,0.280534,0.323292,0.212595
1,RandomForest,0.999180,0.999180,0.999183,0.999181,0.416031,0.416031,0.364817,0.360827


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5650    0.2770    0.3717       361
           2     0.5043    0.8427    0.6310       553
           3     0.6050    0.2353    0.3388       306

    accuracy                         0.5230      1220
   macro avg     0.5581    0.4517    0.4472      1220
weighted avg     0.5475    0.5230    0.4810      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2381    0.0955    0.1364       157
           2     0.4502    0.7637    0.5665       237
           3     0.2542    0.1154    0.1587       130

    accuracy                         0.4027       524
   macro avg     0.3142    0.3249    0.2872       524
weighted avg     0.3381    0.4027    0.3365       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.552214,0.417860,0.444656,0.444656,0.395464,0.350572
1,RandomForest,0.999180,0.999180,0.999183,0.999181,0.412214,0.412214,0.365476,0.363459
2,Adaboost,0.522951,0.522951,0.547534,0.481007,0.402672,0.402672,0.338055,0.336464


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9542    0.8089    0.8756       361
           2     0.7954    0.9982    0.8853       553
           3     0.9864    0.7092    0.8251       306

    accuracy                         0.8697      1220
   macro avg     0.9120    0.8387    0.8620      1220
weighted avg     0.8903    0.8697    0.8673      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2874    0.1592    0.2049       157
           2     0.4724    0.7595    0.5825       237
           3     0.3036    0.1308    0.1828       130

    accuracy                         0.4237       524
   macro avg     0.3545    0.3498    0.3234       524
weighted avg     0.3751    0.4237    0.3702       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.552214,0.417860,0.444656,0.444656,0.395464,0.350572
1,RandomForest,0.999180,0.999180,0.999183,0.999181,0.412214,0.412214,0.365476,0.363459
2,Adaboost,0.522951,0.522951,0.547534,0.481007,0.402672,0.402672,0.338055,0.336464
3,GBM,0.869672,0.869672,0.890296,0.867329,0.423664,0.423664,0.375091,0.370217


**SVM Model**

In [28]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [29]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [30]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [31]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3718    0.5263    0.4358       361
           2     0.5636    0.2803    0.3744       553
           3     0.3502    0.4967    0.4108       306

    accuracy                         0.4074      1220
   macro avg     0.4286    0.4344    0.4070      1220
weighted avg     0.4534    0.4074    0.4017      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3057    0.4459    0.3627       157
           2     0.4771    0.2194    0.3006       237
           3     0.2688    0.3846    0.3165       130

    accuracy                         0.3282       524
   macro avg     0.3505    0.3500    0.3266       524
weighted avg     0.3740    0.3282    0.3231       524



In [32]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.395902,0.395902,0.639916,0.326736,0.280534,0.280534,0.323292,0.212595
1,RandomForest,0.999180,0.999180,0.999183,0.999181,0.416031,0.416031,0.364817,0.360827
2,SVM,0.407377,0.407377,0.453351,0.401693,0.328244,0.328244,0.374049,0.323128


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4827    0.5014    0.4918       361
           2     0.5765    0.7016    0.6330       553
           3     0.5116    0.2876    0.3682       306

    accuracy                         0.5385      1220
   macro avg     0.5236    0.4969    0.4977      1220
weighted avg     0.5325    0.5385    0.5248      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3373    0.3567    0.3467       157
           2     0.4861    0.5907    0.5333       237
           3     0.2286    0.1231    0.1600       130

    accuracy                         0.4046       524
   macro avg     0.3507    0.3568    0.3467       524
weighted avg     0.3776    0.4046    0.3848       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.552214,0.417860,0.444656,0.444656,0.395464,0.350572
1,RandomForest,0.999180,0.999180,0.999183,0.999181,0.412214,0.412214,0.365476,0.363459
2,Adaboost,0.522951,0.522951,0.547534,0.481007,0.402672,0.402672,0.338055,0.336464
3,GBM,0.869672,0.869672,0.890296,0.867329,0.423664,0.423664,0.375091,0.370217
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.538525,0.538525,0.532474,0.524795,0.404580,0.404580,0.377646,0.384808


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3171    0.6676    0.4300       361
           2     0.4881    0.2604    0.3396       553
           3     0.3030    0.1634    0.2123       306

    accuracy                         0.3566      1220
   macro avg     0.3694    0.3638    0.3273      1220
weighted avg     0.3911    0.3566    0.3344      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2975    0.6178    0.4017       157
           2     0.4697    0.2616    0.3360       237
           3     0.2576    0.1308    0.1735       130

    accuracy                         0.3359       524
   macro avg     0.3416    0.3367    0.3037       524
weighted avg     0.3655    0.3359    0.3154       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.502459,0.502459,0.552214,0.417860,0.444656,0.444656,0.395464,0.350572
1,RandomForest,0.999180,0.999180,0.999183,0.999181,0.412214,0.412214,0.365476,0.363459
2,Adaboost,0.522951,0.522951,0.547534,0.481007,0.402672,0.402672,0.338055,0.336464
3,GBM,0.869672,0.869672,0.890296,0.867329,0.423664,0.423664,0.375091,0.370217
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.538525,0.538525,0.532474,0.524795,0.404580,0.404580,0.377646,0.384808
6,GNB,0.356557,0.356557,0.391099,0.334426,0.335878,0.335878,0.365492,0.315369
